# 4교시 2. CNN과 전이학습

## 1. 소규모 데이터셋으로 만드는 강력한 학습 모델

### 치매 환자의 뇌인지 일반인의 뇌인지 예측하기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

# 깃허브에 준비된 데이터를 가져옵니다.
!git clone https://github.com/taehojo/data-ch20.git

# 학습셋의 변형을 설정하는 부분입니다. 
train_datagen = ImageDataGenerator(rescale=1./255,          # 픽셀의 값을 0.0~1.0사이로 정규화합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1)   # 마찬가지로 위, 아래로 이동합니다.

train_generator = train_datagen.flow_from_directory(
       './data-ch20/train',   # 학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 테스트셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(
       './data-ch20/test',   # 테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 앞서 배운 CNN 모델을 만들어 적용해 보겠습니다.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 모델의 실행 옵션을 설정합니다.
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 최적화를 위한 설정 구간입니다.
modelpath="./data-ch20/MNIST_CNN.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델을 실행합니다.
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=30, 
                    verbose=1, 
                    callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(test_generator)[1]))


Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


Cloning into 'data-ch20'...


Epoch 1/30
32/32 [==============================] - ETA: 0s - loss: 0.7001 - accuracy: 0.5625
Epoch 1: val_loss improved from inf to 0.68770, saving model to ./data-ch20\MNIST_CNN.hdf5
32/32 [==============================] - 5s 124ms/step - loss: 0.7001 - accuracy: 0.5625 - val_loss: 0.6877 - val_accuracy: 0.5000
Epoch 2/30
 1/32 [..............................] - ETA: 2s - loss: 0.6853 - accuracy: 0.6000

C:\Users\taehj\anaconda3\envs\tf24\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


32/32 [==============================] - ETA: 0s - loss: 0.6973 - accuracy: 0.5375
Epoch 2: val_loss did not improve from 0.68770
32/32 [==============================] - 2s 76ms/step - loss: 0.6973 - accuracy: 0.5375 - val_loss: 0.6889 - val_accuracy: 0.5000
Epoch 3/30
32/32 [==============================] - ETA: 0s - loss: 0.7058 - accuracy: 0.5437
Epoch 3: val_loss improved from 0.68770 to 0.68666, saving model to ./data-ch20\MNIST_CNN.hdf5
32/32 [==============================] - 2s 71ms/step - loss: 0.7058 - accuracy: 0.5437 - val_loss: 0.6867 - val_accuracy: 0.5333
Epoch 4/30
32/32 [==============================] - ETA: 0s - loss: 0.6911 - accuracy: 0.5437
Epoch 4: val_loss improved from 0.68666 to 0.68090, saving model to ./data-ch20\MNIST_CNN.hdf5
32/32 [==============================] - 2s 75ms/step - loss: 0.6911 - accuracy: 0.5437 - val_loss: 0.6809 - val_accuracy: 0.5000
Epoch 5/30
32/32 [==============================] - ETA: 0s - loss: 0.6866 - accuracy: 0.5875
Epoch 5:

## 2. 전이 학습으로 모델 성능 극대화하기

### 실습: 전이 학습 실습하기

<img src="../data/img/04-04.png" width="400"/>

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

# 학습셋의 변형을 설정하는 부분입니다. 
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1)   # 마찬가지로 위, 아래로 이동합니다.

train_generator = train_datagen.flow_from_directory(
       './data-ch20/train',
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 테스트셋의 정규화를 설정합니다.
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
       './data-ch20/test',
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# VGG16 모델을 불러옵니다.
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False

# 우리의 모델을 설정합니다.
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(1, activation='sigmoid'))

# 모델의 실행 옵션을 설정합니다. 
finetune_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델을 실행합니다.
history = finetune_model.fit(
       train_generator,
       epochs=30,
       validation_data=test_generator,
       validation_steps=10, 
       callbacks=[early_stopping_callback])

# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (finetune_model.evaluate(test_generator)[1]))


Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/30
32/32 [==============================] - 7s 191ms/step - loss: 0.9440 - accuracy: 0.5500 - val_loss: 0.6437 - val_accuracy: 0.5600
Epoch 2/30
32/32 [==============================] - 6s 192ms/step - loss: 0.5907 - accuracy: 0.6812 - val_loss: 0.4814 - val_accuracy: 0.7800
Epoch 3/30
32/32 [==============================] - 6s 188ms/step - loss: 0.5075 - accuracy: 0.6938 - val_loss: 0.3831 - val_accuracy: 0.9000
Epoch 4/30
32/32 [==============================] - 6s 194ms/step - loss: 0.3878 - accuracy: 0.8313 - val_loss: 0.3947 - val_accuracy: 0.8600
Epoch 5/30
32/32 [==============================] - 6s 200ms/step - loss: 0.3696 - accuracy: 0.8438 - val_loss: 0.2996 - val_accuracy: 0.9200
Epoch 6/30
32/32 [==============================] - 6s 203ms/step - loss: 0.3414 - accuracy: 0.8562 - val_loss: 0.3019 - val_accuracy: 0.8600
Epoch 7/30
32/32 [==============================] - 6s 197ms/step 